In [2]:
# Import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Import evaluation libraries
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Import model libraries
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

In [3]:
# Import the train set and test set
train_data = pd.read_csv("final_train.csv", delimiter=",")
test_data = pd.read_csv("final_test.csv", delimiter=",")

# Drop the ID column
train_data = train_data.drop('ID', axis=1)

# Sort the dataset
train_data = train_data.iloc[np.random.permutation(len(train_data))]
train_data = train_data.reset_index(drop=True)

In [4]:
# Select the best features for binary and multiclass tasks

array = np.asarray(train_data)

# X,Y are the splits between features and labels used to evaluate SelectKBest
X = array[:,0:train_data.shape[1]-1]
X = np.asarray(X)
Y = array[:,train_data.shape[1]-1]
Y = np.asarray(Y)

# Evaluate the features with a chi2 test by using SelectKBest
bin_chi2_test = SelectKBest(score_func=chi2, k=28)
fit = bin_chi2_test.fit(X,Y)
bin_feat = train_data.columns[bin_chi2_test.get_support(indices=True)]

v_chi2_test = SelectKBest(score_func=chi2, k=27)
fit = v_chi2_test.fit(X,Y)
v_feat = train_data.columns[v_chi2_test.get_support(indices=True)]

In [5]:
def features_labels_split(data):
    
    features = data[bin_feat]
    labels = data['Product']
    return features, labels

In [6]:
def standardize_features(features):

    scaler = MinMaxScaler().fit(features)
    features = scaler.transform(features)
    return features

In [7]:
def train_test_validation_split(features, labels, val_samples, test_samples):

    features = np.asarray(features)
    features = standardize_features(features)
    labels =np.asarray(labels)
    
    X_test = features[0:test_samples]
    y_test = labels[0:test_samples]

    X_val = features[test_samples:test_samples + val_samples]
    y_val = labels[test_samples:test_samples + val_samples]

    X_train = features[test_samples + val_samples:]
    y_train = labels[test_samples + val_samples:]

    return X_train, X_test, X_val, y_train, y_test, y_val

In [23]:
train_feat.shape

(9567, 28)

In [75]:
# train_feat, train_label are the splits of train_data between features and labels
train_feat, train_label = features_labels_split(train_data)

# bin_train_label is the same of train_label but BINARY
bin_train_label = []
 
for i in range(0, len(train_label)):
    if(train_label[i] == 0):
        bin_train_label.append(0)
    else:
        bin_train_label.append(1)
bin_train_label = np.asarray(bin_train_label)
#print(train_label)
#print(bin_train_label)
num_val_samples = 950
num_test_samples = 950

# bin_X_train, bin_X_test, bin_X_val, bin_y_train, bin_y_test, bin_y_val are the splits of train_feat and bin_train_label
bin_X_train, bin_X_test, bin_X_val, bin_y_train, bin_y_test, bin_y_val = train_test_validation_split(train_feat, bin_train_label, num_val_samples, num_test_samples)

In [76]:
bin_X_train.shape

(7667, 28)

In [77]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam

In [78]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [85]:
neurons = 64
dropout = 0.5
batch_size = 64         
epochs = 5000

output_activation_function = 'sigmoid'
activation_function = 'relu'

loss = 'binary_crossentropy'
learning_rate = 0.001
optimizer= Adam(learning_rate)

def build_model(inputs, output_size, neurons, activ_func=activation_function,
                dropout = dropout, loss=loss, optimizer=optimizer):
    
    model = Sequential()
    model.add(Dense(neurons, input_shape=(bin_X_train.shape[1],)))
    model.add(Activation(activation_function))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size, activation=output_activation_function))    
    model.compile(loss=loss, optimizer=optimizer, metrics=[f1])
    return model

In [86]:
# initialise model architecture
output_size = 1
eth_model = build_model(bin_X_train, output_size=output_size, neurons=neurons)

In [ ]:
# train model on data
eth_history = eth_model.fit(bin_X_train, bin_y_train,
                            epochs=epochs, batch_size=batch_size,
                            verbose=1, validation_data=(bin_X_val, bin_y_val),
                            shuffle=False)

Train on 7667 samples, validate on 950 samples
Epoch 1/5000
7667/7667 [==============================] - 8s 1ms/step - loss: 0.6653 - f1: 0.4741 - val_loss: 0.6304 - val_f1: 0.5341
Epoch 2/5000
7667/7667 [==============================] - 3s 344us/step - loss: 0.6366 - f1: 0.5599 - val_loss: 0.6154 - val_f1: 0.5661
Epoch 3/5000
7667/7667 [==============================] - 3s 334us/step - loss: 0.6296 - f1: 0.5797 - val_loss: 0.6095 - val_f1: 0.5731
Epoch 4/5000
7667/7667 [==============================] - 3s 356us/step - loss: 0.6222 - f1: 0.5893 - val_loss: 0.6031 - val_f1: 0.5775
Epoch 5/5000
7667/7667 [==============================] - 2s 308us/step - loss: 0.6161 - f1: 0.5901 - val_loss: 0.5994 - val_f1: 0.5912
Epoch 6/5000
7667/7667 [==============================] - 3s 378us/step - loss: 0.6133 - f1: 0.5979 - val_loss: 0.5960 - val_f1: 0.5874
Epoch 7/5000
7667/7667 [==============================] - 3s 382us/step - loss: 0.6118 - f1: 0.5949 - val_loss: 0.5936 - val_f1: 0.5846
Epo

7667/7667 [==============================] - 1s 141us/step - loss: 0.5706 - f1: 0.6404 - val_loss: 0.5801 - val_f1: 0.6120
Epoch 61/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5718 - f1: 0.6386 - val_loss: 0.5801 - val_f1: 0.6126
Epoch 62/5000
7667/7667 [==============================] - 1s 140us/step - loss: 0.5700 - f1: 0.6375 - val_loss: 0.5802 - val_f1: 0.6164
Epoch 63/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5703 - f1: 0.6342 - val_loss: 0.5799 - val_f1: 0.6239
Epoch 64/5000
7667/7667 [==============================] - 1s 140us/step - loss: 0.5707 - f1: 0.6394 - val_loss: 0.5798 - val_f1: 0.6217
Epoch 65/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5718 - f1: 0.6346 - val_loss: 0.5797 - val_f1: 0.6180
Epoch 66/5000
7667/7667 [==============================] - 1s 141us/step - loss: 0.5719 - f1: 0.6366 - val_loss: 0.5798 - val_f1: 0.6100
Epoch 67/5000
7667/7667 [==============================

Epoch 120/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5568 - f1: 0.6587 - val_loss: 0.5887 - val_f1: 0.6258
Epoch 121/5000
7667/7667 [==============================] - 1s 141us/step - loss: 0.5553 - f1: 0.6563 - val_loss: 0.5888 - val_f1: 0.6327
Epoch 122/5000
7667/7667 [==============================] - 1s 137us/step - loss: 0.5569 - f1: 0.6625 - val_loss: 0.5881 - val_f1: 0.6251
Epoch 123/5000
7667/7667 [==============================] - 1s 135us/step - loss: 0.5577 - f1: 0.6583 - val_loss: 0.5879 - val_f1: 0.6283
Epoch 124/5000
7667/7667 [==============================] - 1s 136us/step - loss: 0.5564 - f1: 0.6585 - val_loss: 0.5895 - val_f1: 0.6294
Epoch 125/5000
7667/7667 [==============================] - 1s 135us/step - loss: 0.5549 - f1: 0.6542 - val_loss: 0.5899 - val_f1: 0.6236
Epoch 126/5000
7667/7667 [==============================] - 1s 135us/step - loss: 0.5529 - f1: 0.6565 - val_loss: 0.5897 - val_f1: 0.6263
Epoch 127/5000
7667/7667 [========

7667/7667 [==============================] - 1s 136us/step - loss: 0.5477 - f1: 0.6629 - val_loss: 0.5905 - val_f1: 0.6276
Epoch 180/5000
7667/7667 [==============================] - 1s 137us/step - loss: 0.5472 - f1: 0.6652 - val_loss: 0.5914 - val_f1: 0.6281
Epoch 181/5000
7667/7667 [==============================] - 1s 143us/step - loss: 0.5450 - f1: 0.6682 - val_loss: 0.5918 - val_f1: 0.6326
Epoch 182/5000
7667/7667 [==============================] - 1s 134us/step - loss: 0.5502 - f1: 0.6644 - val_loss: 0.5918 - val_f1: 0.6313
Epoch 183/5000
7667/7667 [==============================] - 1s 154us/step - loss: 0.5461 - f1: 0.6639 - val_loss: 0.5902 - val_f1: 0.6353
Epoch 184/5000
7667/7667 [==============================] - 1s 134us/step - loss: 0.5452 - f1: 0.6791 - val_loss: 0.5911 - val_f1: 0.6264
Epoch 185/5000
7667/7667 [==============================] - 1s 130us/step - loss: 0.5418 - f1: 0.6775 - val_loss: 0.5910 - val_f1: 0.6250
Epoch 186/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 135us/step - loss: 0.5387 - f1: 0.6760 - val_loss: 0.5947 - val_f1: 0.6288
Epoch 239/5000
7667/7667 [==============================] - 1s 136us/step - loss: 0.5359 - f1: 0.6765 - val_loss: 0.5961 - val_f1: 0.6292
Epoch 240/5000
7667/7667 [==============================] - 1s 134us/step - loss: 0.5414 - f1: 0.6723 - val_loss: 0.5938 - val_f1: 0.6350
Epoch 241/5000
7667/7667 [==============================] - 1s 136us/step - loss: 0.5407 - f1: 0.6735 - val_loss: 0.5944 - val_f1: 0.6279
Epoch 242/5000
7667/7667 [==============================] - 1s 134us/step - loss: 0.5372 - f1: 0.6739 - val_loss: 0.5960 - val_f1: 0.6323
Epoch 243/5000
7667/7667 [==============================] - 1s 134us/step - loss: 0.5359 - f1: 0.6773 - val_loss: 0.5952 - val_f1: 0.6451
Epoch 244/5000
7667/7667 [==============================] - 1s 135us/step - loss: 0.5388 - f1: 0.6758 - val_loss: 0.5954 - val_f1: 0.6365
Epoch 245/5000
7667/7667 [=======================

...

7667/7667 [==============================] - 1s 131us/step - loss: 0.5257 - f1: 0.6896 - val_loss: 0.6000 - val_f1: 0.6331
Epoch 416/5000
7667/7667 [==============================] - 1s 133us/step - loss: 0.5221 - f1: 0.6932 - val_loss: 0.6010 - val_f1: 0.6334
Epoch 417/5000
7667/7667 [==============================] - 1s 131us/step - loss: 0.5251 - f1: 0.6895 - val_loss: 0.5997 - val_f1: 0.6409
Epoch 418/5000
7667/7667 [==============================] - 1s 131us/step - loss: 0.5246 - f1: 0.6839 - val_loss: 0.5967 - val_f1: 0.6421
Epoch 419/5000
7667/7667 [==============================] - 1s 131us/step - loss: 0.5230 - f1: 0.6910 - val_loss: 0.6012 - val_f1: 0.6370
Epoch 420/5000
7667/7667 [==============================] - 1s 131us/step - loss: 0.5310 - f1: 0.6874 - val_loss: 0.6010 - val_f1: 0.6420
Epoch 421/5000
7667/7667 [==============================] - 1s 131us/step - loss: 0.5257 - f1: 0.6872 - val_loss: 0.6007 - val_f1: 0.6273
Epoch 422/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 145us/step - loss: 0.5223 - f1: 0.6939 - val_loss: 0.6003 - val_f1: 0.6417
Epoch 475/5000
7667/7667 [==============================] - 1s 142us/step - loss: 0.5217 - f1: 0.6952 - val_loss: 0.6018 - val_f1: 0.6428
Epoch 476/5000
7667/7667 [==============================] - 1s 141us/step - loss: 0.5229 - f1: 0.6834 - val_loss: 0.6018 - val_f1: 0.6414
Epoch 477/5000
7667/7667 [==============================] - 1s 143us/step - loss: 0.5238 - f1: 0.6922 - val_loss: 0.6019 - val_f1: 0.6500
Epoch 478/5000
7667/7667 [==============================] - 1s 140us/step - loss: 0.5278 - f1: 0.6873 - val_loss: 0.6014 - val_f1: 0.6410
Epoch 479/5000
7667/7667 [==============================] - 1s 137us/step - loss: 0.5250 - f1: 0.6892 - val_loss: 0.6018 - val_f1: 0.6466
Epoch 480/5000
7667/7667 [==============================] - 1s 143us/step - loss: 0.5214 - f1: 0.6910 - val_loss: 0.6032 - val_f1: 0.6404
Epoch 481/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 136us/step - loss: 0.5180 - f1: 0.6890 - val_loss: 0.6019 - val_f1: 0.6448
Epoch 534/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5238 - f1: 0.6913 - val_loss: 0.6008 - val_f1: 0.6463
Epoch 535/5000
7667/7667 [==============================] - 1s 140us/step - loss: 0.5196 - f1: 0.6956 - val_loss: 0.6024 - val_f1: 0.6458
Epoch 536/5000
7667/7667 [==============================] - 1s 138us/step - loss: 0.5201 - f1: 0.6981 - val_loss: 0.5999 - val_f1: 0.6410
Epoch 537/5000
7667/7667 [==============================] - 1s 136us/step - loss: 0.5221 - f1: 0.6953 - val_loss: 0.5982 - val_f1: 0.6463
Epoch 538/5000
7667/7667 [==============================] - 1s 135us/step - loss: 0.5222 - f1: 0.6878 - val_loss: 0.6005 - val_f1: 0.6301
Epoch 539/5000
7667/7667 [==============================] - 1s 142us/step - loss: 0.5185 - f1: 0.6975 - val_loss: 0.6038 - val_f1: 0.6434
Epoch 540/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 139us/step - loss: 0.5214 - f1: 0.6862 - val_loss: 0.5989 - val_f1: 0.6370
Epoch 593/5000
7667/7667 [==============================] - 1s 151us/step - loss: 0.5199 - f1: 0.6892 - val_loss: 0.5992 - val_f1: 0.6358
Epoch 594/5000
7667/7667 [==============================] - 1s 145us/step - loss: 0.5248 - f1: 0.6866 - val_loss: 0.5981 - val_f1: 0.6408
Epoch 595/5000
7667/7667 [==============================] - 1s 142us/step - loss: 0.5217 - f1: 0.6920 - val_loss: 0.5996 - val_f1: 0.6394
Epoch 596/5000
7667/7667 [==============================] - 1s 136us/step - loss: 0.5161 - f1: 0.6968 - val_loss: 0.5998 - val_f1: 0.6384
Epoch 597/5000
7667/7667 [==============================] - 1s 139us/step - loss: 0.5199 - f1: 0.6873 - val_loss: 0.5986 - val_f1: 0.6359
Epoch 598/5000
7667/7667 [==============================] - 1s 145us/step - loss: 0.5241 - f1: 0.6903 - val_loss: 0.5982 - val_f1: 0.6373
Epoch 599/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 145us/step - loss: 0.5162 - f1: 0.6949 - val_loss: 0.5964 - val_f1: 0.6398
Epoch 652/5000
7667/7667 [==============================] - 1s 145us/step - loss: 0.5124 - f1: 0.6929 - val_loss: 0.5973 - val_f1: 0.6433
Epoch 653/5000
7667/7667 [==============================] - 1s 143us/step - loss: 0.5206 - f1: 0.6903 - val_loss: 0.5965 - val_f1: 0.6372
Epoch 654/5000
7667/7667 [==============================] - 1s 153us/step - loss: 0.5199 - f1: 0.6922 - val_loss: 0.5959 - val_f1: 0.6369
Epoch 655/5000
7667/7667 [==============================] - 1s 145us/step - loss: 0.5203 - f1: 0.6891 - val_loss: 0.5975 - val_f1: 0.6427
Epoch 656/5000
7667/7667 [==============================] - 1s 153us/step - loss: 0.5170 - f1: 0.6905 - val_loss: 0.5974 - val_f1: 0.6341
Epoch 657/5000
7667/7667 [==============================] - 1s 143us/step - loss: 0.5177 - f1: 0.6981 - val_loss: 0.5960 - val_f1: 0.6361
Epoch 658/5000
7667/7667 [=======================